In [1]:
import sys
import re
import random
import time
import os

from tqdm import tqdm

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
from collections import Counter
from joypy import joyplot

sys.path.append("../")
from calculus_path_mod.term_engine import *
from calculus_path_mod.reduction_strategy import *
from calculus_path_mod.terms import num_comparison, nat_numbers, arithm_ops, combinators, pairs, logic
from calculus_path_mod.terms.pseudonym import *

from calculus_path_mod.json_serialization import load_terms
from fitter import Fitter, get_common_distributions
from calculus_utils.drawing import draw_steps_displot

# Load Terms filtered LO

In [2]:
terms_LO = load_terms("../tests_11_retests/terms_210_filtered_LO.dat")
# terms_RI = load_terms("../tests_11_retests/terms_210_filtered_RI.dat")

In [5]:
from calculus_path_mod.reduction_strategy import RandomOuterStrategy

ro_strategy = RandomOuterStrategy()


def tree_normalize_with_params(term, is_limited=True, max_count_step_term=400, steps=400, vertices_lim=7_000):
    norm_steps_list = []

    try:
        dict_redexes_indexes = ro_strategy._get_redexes_indexes(term)
    except ValueError:
        dict_redexes_indexes = dict()
    norm_params = {
        "term": term,
        "prev_term_redex": (0, -1),
        "redex_depths": list(dict_redexes_indexes.values()),
        "redex_indexes": list(dict_redexes_indexes.keys()),
        "step_time": 0.0,
    }

    norm_steps_list.append([norm_params, ])
    step_inx = 0
    is_normalizable = len(norm_params["redex_indexes"]) > 0

    while is_normalizable:
        norm_steps_list.append(list())
        iter_indexes = range(len(norm_steps_list[step_inx]))
        iter_indexes = iter_indexes if len(norm_steps_list[step_inx]) < max_count_step_term else random.sample(
            iter_indexes, max_count_step_term)
        for inx in iter_indexes:
            for redex_index in norm_steps_list[step_inx][inx]["redex_indexes"]:
                if is_limited and norm_steps_list[step_inx][inx]["term"].vertices_number > vertices_lim:
                    continue
                start_time = time.process_time_ns()
                reduced_term = norm_steps_list[step_inx][inx]["term"]._update_bound_vars()
                reduced_term = reduced_term._beta_conversion_visual(redex_index)
                end_time = time.process_time_ns()

                try:
                    dict_redexes_indexes = ro_strategy._get_redexes_indexes(reduced_term)
                except ValueError:
                    dict_redexes_indexes = dict()
                norm_params = {
                    "term": reduced_term,
                    "prev_term_redex": (inx, redex_index),
                    "redex_depths": list(dict_redexes_indexes.values()),
                    "redex_indexes": list(dict_redexes_indexes.keys()),
                    "step_time": end_time - start_time,
                }

                norm_steps_list[step_inx + 1].append(norm_params)

        if is_limited and step_inx == steps:
            break

        step_inx += 1
        is_normalizable = False
        for norm_params in norm_steps_list[step_inx]:
            if len(norm_params["redex_indexes"]) > 0:
                is_normalizable = True
                break
    return norm_steps_list

In [4]:
len(terms_LO)

226

In [5]:
def gen_norm_tree_data(terms_list):
    for inx_term in tqdm(range(223, len(terms_list))):
        try:
            term_steps_data = tree_normalize_with_params(terms_list[inx_term], max_count_step_term=200, vertices_lim=3_000, steps=40)
        except:
            continue

        tsd_df = pd.DataFrame(columns=[
            "vertices_pre", "redexes_pre", "height_pre", "width_pre",
            "vertices_post", "redexes_post", "height_post", "width_post",
            "redex_depth", "redex_index", "step_time"
        ])

        for inx in range(1, len(term_steps_data)):
            for post_term_data in term_steps_data[inx]:
                prev_term_redex_info = post_term_data["prev_term_redex"]
                pre_term_data = term_steps_data[inx - 1][prev_term_redex_info[0]]

                term_data_dict = {
                    "vertices_pre": [pre_term_data["term"].vertices_number],
                    "redexes_pre": [len(pre_term_data["term"].redexes)],
                    "height_pre": [pre_term_data["term"].term_height],
                    "width_pre": [pre_term_data["term"].term_width],

                    "vertices_post": [post_term_data["term"].vertices_number],
                    "redexes_post": [len(post_term_data["term"].redexes)],
                    "height_post": [post_term_data["term"].term_height],
                    "width_post": [post_term_data["term"].term_width],

                    "redex_depth": [pre_term_data["term"].redex_depth(prev_term_redex_info[1])],
                    "redex_index": [prev_term_redex_info[1]],
                    "step_time": [post_term_data["step_time"]],
                }

                term_data_dict["vertices_diff"] = [
                    term_data_dict["vertices_pre"][0] - term_data_dict["vertices_post"][0]]
                term_data_dict["redexes_diff"] = [term_data_dict["redexes_pre"][0] - term_data_dict["redexes_post"][0]]
                term_data_dict["height_diff"] = [term_data_dict["height_pre"][0] - term_data_dict["height_post"][0]]
                term_data_dict["width_diff"] = [term_data_dict["width_pre"][0] - term_data_dict["width_post"][0]]

                tsd_df = pd.concat([tsd_df, pd.DataFrame(term_data_dict)], ignore_index=True).reset_index(drop=True)

        tsd_df.to_csv(f"./collected_data/tree_reduction_t_{inx_term}.csv")

In [6]:
gen_norm_tree_data(terms_LO)

100%|██████████| 3/3 [00:23<00:00,  7.74s/it]


# Fix the data

In [9]:
path = "./collected_data/"
dir_list = os.listdir(path)
print(dir_list)

['tree_reduction_t_0.csv', 'tree_reduction_t_1.csv', 'tree_reduction_t_10.csv', 'tree_reduction_t_100.csv', 'tree_reduction_t_101.csv', 'tree_reduction_t_102.csv', 'tree_reduction_t_103.csv', 'tree_reduction_t_104.csv', 'tree_reduction_t_105.csv', 'tree_reduction_t_106.csv', 'tree_reduction_t_107.csv', 'tree_reduction_t_108.csv', 'tree_reduction_t_109.csv', 'tree_reduction_t_11.csv', 'tree_reduction_t_110.csv', 'tree_reduction_t_111.csv', 'tree_reduction_t_112.csv', 'tree_reduction_t_113.csv', 'tree_reduction_t_114.csv', 'tree_reduction_t_115.csv', 'tree_reduction_t_116.csv', 'tree_reduction_t_117.csv', 'tree_reduction_t_118.csv', 'tree_reduction_t_119.csv', 'tree_reduction_t_12.csv', 'tree_reduction_t_120.csv', 'tree_reduction_t_121.csv', 'tree_reduction_t_122.csv', 'tree_reduction_t_123.csv', 'tree_reduction_t_124.csv', 'tree_reduction_t_125.csv', 'tree_reduction_t_126.csv', 'tree_reduction_t_127.csv', 'tree_reduction_t_128.csv', 'tree_reduction_t_129.csv', 'tree_reduction_t_13.csv',

In [10]:
pd.read_csv(path + "/" + dir_list[0]).head()

,Unnamed: 0,vertices_pre,redexes_pre,height_pre,width_pre,vertices_post,redexes_post,height_post,width_post,redex_depth,redex_index,step_time,vertices_diff,redexes_diff,height_diff,width_diff
0,0,53,3,15,21,7,0,3,2,1,1,-161100,46.0,3.0,12.0,19.0
1,1,53,3,15,21,50,2,15,20,4,12,-129837,3.0,1.0,0.0,1.0
2,2,53,3,15,21,30,2,9,11,5,30,-141100,23.0,1.0,6.0,10.0
3,3,50,2,15,20,7,0,3,2,1,1,-110351,43.0,2.0,12.0,18.0
4,4,50,2,15,20,27,1,8,10,5,27,-126194,23.0,1.0,7.0,10.0


In [14]:
for file_name in dir_list:
    file_path = path + "/" + file_name
    data = pd.read_csv(file_path)
    data = data[[ri != -1 for ri in data["redex_index"]]]
    data = data.drop('Unnamed: 0', axis=1)
    data["step_time"] = np.array(data["step_time"]) * -1
    data.to_csv(file_path, index=False)

# just example of tree reduction

In [3]:
terms_LO[0]

In [6]:
term_0_norm_tree = tree_normalize_with_params(terms_LO[0], vertices_lim=3_000, steps=30)

In [7]:
len(term_0_norm_tree)

4

In [8]:
for inx, rec in enumerate(term_0_norm_tree):
    print(f"{inx}")
    for iinx, irec in enumerate(rec):
        print(iinx, irec)
    print("\n\n\n")

0
0 {'term': <calculus_path_mod.term_engine.Application object at 0x00000232C0F06390>, 'prev_term_redex': (0, -1), 'redex_depths': [1, 4, 5], 'redex_indexes': [1, 12, 30], 'step_time': 0.0}




1
0 {'term': <calculus_path_mod.term_engine.Abstraction object at 0x00000232C1550A50>, 'prev_term_redex': (0, 1), 'redex_depths': [], 'redex_indexes': [], 'step_time': 0}
1 {'term': <calculus_path_mod.term_engine.Application object at 0x00000232C1551090>, 'prev_term_redex': (0, 12), 'redex_depths': [1, 5], 'redex_indexes': [1, 27], 'step_time': 0}
2 {'term': <calculus_path_mod.term_engine.Application object at 0x00000232C1550E10>, 'prev_term_redex': (0, 30), 'redex_depths': [1, 4], 'redex_indexes': [1, 12], 'step_time': 0}




2
0 {'term': <calculus_path_mod.term_engine.Abstraction object at 0x00000232C10B4D50>, 'prev_term_redex': (1, 1), 'redex_depths': [], 'redex_indexes': [], 'step_time': 0}
1 {'term': <calculus_path_mod.term_engine.Application object at 0x00000232C154EF10>, 'prev_term_redex'